In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']= '3'

import tensorflow as tf
tf.autograph.set_verbosity(0)

In [3]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
os.chdir('/content/gdrive/My Drive/FYP/Speaker/Contrastive')
from siamese import Siamese as Contrastive
os.chdir('/content/gdrive/My Drive/FYP/Speaker/Triplet')
from siamese import Siamese as Triplet
os.chdir('/content/gdrive/My Drive/FYP/Speaker/Tests')

In [7]:
LSTM_CONTRASTIVE = 'Contrastive/Model/LSTM/'
CONV1D_CONTRASTIVE = 'Contrastive/Model/Conv1D/'
CONV2D_CONTRASTIVE = 'Contrastive/Model/Conv2D/'
LSTM_TRIPLET = 'Triplet/Model/LSTM/'
CONV1D_TRIPLET = 'Triplet/Model/Conv1D/'
CONV2D_TRIPLET = 'Triplet/Model/Conv2D/'
DATA_PATH = '../Data/Data/'

In [8]:
W = 300
H = 128

In [9]:
print('Loading data')
with open(DATA_PATH + 'test.pickle', 'rb') as f:
    X_test, s_test = pickle.load(f)

Loading data


In [13]:
def test(X, s, N, k, m1, m2, m3, m4, m5, m6):
    a1, a2, a3, a4, a5, a6 = 0, 0, 0, 0, 0, 0

    for i in range(k):
        p, y = get_batch(X, s, N)

        p1 = m1.predict(p)
        if np.argmin(y) == np.argmin(p1):
            a1 += 1

        p2 = m2.predict(p)
        if np.argmin(y) == np.argmin(p2):
            a2 += 1

        p3 = m3.predict(p)
        if np.argmin(y) == np.argmin(p3):
            a3 += 1

        p4 = m4.predict(p)
        if np.argmin(y) == np.argmin(p4):
            a4 += 1

        p5 = m5.predict(p)
        if np.argmin(y) == np.argmin(p5):
            a5 += 1

        p6 = m6.predict(p)
        if np.argmin(y) == np.argmin(p6):
            a6 += 1


    a1 *= (100/k)
    a2 *= (100/k)
    a3 *= (100/k)
    a4 *= (100/k)
    a5 *= (100/k)
    a6 *= (100/k)
    print(N, ':', a1, a2, a3, a4, a5, a6)
    return a1, a2, a3, a4, a5, a6

In [11]:
def get_pairs(X, s, sim):
    speakers = list(s.keys())
    num = len(speakers)
    
    if sim:
        s1 = np.random.randint(0, num)
        l1, h1 = s[speakers[s1]]
        
        i1_1 = np.random.randint(l1, h1+1) - l1
        i1_2 = np.random.randint(l1, h1+1) - l1
        while i1_2 == i1_1:
            i1_2 = np.random.randint(l1, h1+1) - l1
            
        p1 = abs(X[s1][i1_1])
        p2 = abs(X[s1][i1_2])
        y = 0.
        
    else:
        s1 = np.random.randint(0, num)
        s2 = np.random.randint(0, num)
        while s1 == s2:
            s2 = np.random.randint(0, num)
            
        l1, h1 = s[speakers[s1]]
        l2, h2 = s[speakers[s2]]
        
        i1 = np.random.randint(l1, h1+1) - l1
        i2 = np.random.randint(l2, h2+1) - l2
        
        p1 = abs(X[s1][i1])
        p2 = abs(X[s2][i2])
        y = 1.
    
    return p1.reshape(W, H, 1), p2.reshape(W, H, 1), y

def get_batch(X, s, batch_size):
    pair1 = []
    pair2 = []
    labels = []
    
    for _ in range(1):
        p1, p2, y = get_pairs(X, s, True)
        pair1.append(p1)
        pair2.append(p2)
        labels.append(y)
        
    for _ in range(batch_size - 1):
        p1, p2, y = get_pairs(X, s, False)
        pair1.append(p1)
        pair2.append(p2)
        labels.append(y)
        
    pair1, pair2, labels = np.array(pair1), np.array(pair2), np.array(labels)
    _ = np.random.permutation(len(pair1))
    return [pair1[_], pair2[_]], labels[_]

In [12]:
m1 = Contrastive((W, H, 1), name='lstm', model_path=LSTM_CONTRASTIVE+'lstm_50.h5', margin=0.5)
m2 = Contrastive((W, H, 1), name='conv_1d', model_path=CONV1D_CONTRASTIVE+'conv1d_50.h5', margin=0.5)
m3 = Contrastive((W, H, 1), name='conv_2d', model_path=CONV2D_CONTRASTIVE+'conv2d_50.h5', margin=0.5)

m4 = Triplet((W, H, 1), name='lstm', model_path=LSTM_TRIPLET+'lstm_50.h5', margin=1)
m5 = Triplet((W, H, 1), name='conv_1d', model_path=CONV1D_TRIPLET+'conv1d_50.h5', margin=1)
m6 = Triplet((W, H, 1), name='conv_2d', model_path=CONV2D_TRIPLET+'conv2d_50.h5', margin=1)

In [14]:
lstm_con, conv1d_con, conv2d_con, lstm_tri, conv1d_tri, conv2d_tri = [], [], [], [], [], []
for i in range(2, 33):
    a1, a2, a3, a4, a5, a6 = test(X_test, s_test, i, 100, m1.model, m2.model, m3.model, m4.model, m5.model, m6.model)
    lstm_con.append(a1)
    conv1d_con.append(a2)
    conv2d_con.append(a3)
    lstm_tri.append(a4)
    conv1d_tri.append(a5)
    conv2d_tri.append(a6)

2 : 86.0 89.0 93.0 84.0 91.0 81.0
3 : 87.0 88.0 93.0 78.0 92.0 76.0
4 : 80.0 81.0 84.0 79.0 79.0 66.0
5 : 69.0 78.0 76.0 66.0 80.0 70.0
6 : 60.0 79.0 74.0 54.0 73.0 54.0
7 : 57.0 68.0 75.0 59.0 65.0 48.0
8 : 55.0 65.0 71.0 55.0 70.0 59.0
9 : 58.0 66.0 71.0 63.0 71.0 55.0
10 : 46.0 56.0 66.0 49.0 59.0 44.0
11 : 50.0 62.0 60.0 47.0 63.0 48.0
12 : 50.0 56.0 66.0 47.0 61.0 40.0
13 : 52.0 61.0 64.0 50.0 58.0 37.0
14 : 34.0 56.0 59.0 37.0 58.0 42.0
15 : 30.0 53.0 58.0 40.0 55.0 36.0
16 : 32.0 53.0 52.0 39.0 60.0 37.0
17 : 31.0 42.0 46.0 39.0 54.0 35.0
18 : 38.0 49.0 59.0 34.0 48.0 37.0
19 : 31.0 55.0 50.0 28.0 36.0 23.0
20 : 34.0 47.0 51.0 34.0 47.0 38.0
21 : 35.0 50.0 52.0 31.0 53.0 34.0
22 : 22.0 39.0 41.0 32.0 53.0 31.0
23 : 32.0 40.0 51.0 28.0 37.0 27.0
24 : 33.0 43.0 44.0 34.0 45.0 27.0
25 : 33.0 40.0 42.0 32.0 43.0 31.0
26 : 37.0 40.0 43.0 23.0 36.0 23.0
27 : 30.0 43.0 41.0 19.0 46.0 24.0
28 : 23.0 40.0 45.0 25.0 52.0 23.0
29 : 24.0 38.0 40.0 27.0 39.0 25.0
30 : 27.0 40.0 47.0 18.0 36.

In [16]:
result = (lstm_con, conv1d_con, conv2d_con, lstm_tri, conv1d_tri, conv2d_tri)
f = open('/content/gdrive/My Drive/FYP/Speaker/Tests/result.pickle', 'wb')
pickle.dump(result, f)